In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from numpy import load

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

In [ ]:
X_train=[]
y_train=[]
X_val=[]
y_val=[]

os.chdir('/content/drive/MyDrive/생체인증보안/02_face_training')
# for i in os.listdir():
#     img = cv2.imread(i)   
#     img = cv2.resize(img,(160,160))
#     if(i[5:9]=='0003'):
#         X_val.append(img)
#         y_val.append(i[0:4])
#     else:
#         X_train.append(img)
#         y_train.append(i[0:4])

for i in os.listdir():
    img = cv2.imread(i)   
    img = cv2.resize(img,(160,160))
    if(i[5:9]=='0003'):
        X_val.append(img)
        y_val.append(i[0:4])
        X_train.append(img)
        y_train.append(i[0:4])
    else:
        X_train.append(img)
        y_train.append(i[0:4])

In [ ]:
X_test = []
y_test = []

os.chdir('/content/drive/MyDrive/생체인증보안/02_face_test')
for i in os.listdir():
      img = cv2.imread(i)   
      img = cv2.resize(img,(160,160))
      X_test.append(img)
      y_test.append(i.rstrip('.bmp'))

In [ ]:
print ("Shape of an image in X_train: ", X_train[0].shape)
print ("Shape of an image in X_train: ", X_val[0].shape)
print ("Shape of an image in X_test: ", X_test[0].shape)
print("Total categories: ", len(np.unique(y_train)))
print("Total categories: ", len(np.unique(y_val)))
print("Total categories: ", len(np.unique(y_test)))

In [ ]:
le = preprocessing.LabelEncoder()

y_train = le.fit_transform(y_train)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=350)
y_train = np.array(y_train)

y_val = le.fit_transform(y_val)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=350)
y_val = np.array(y_val)

X_train = np.array(X_train)
X_test = np.array(X_test)
X_val=np.array(X_val)

In [ ]:
from numpy import asarray
print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)

print("X_val Shape: ", X_val.shape)
print("y_val Shape: ", y_val.shape)

print("y_test Shape: ", X_test.shape)

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#         rescale=1./255,
#         zoom_range=0.1,
#         brightness_range=[0.2,1.0],
#         fill_mode='constant')

In [ ]:
from imgaug import augmenters as iaa

seq = iaa.Sequential([
    # blur images with a sigma of 0 to 0.5
    iaa.GaussianBlur(sigma=(0, 0.5)),
    iaa.Affine(
        # scale images to 90-110% of their size, individually per axis
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        # translate by -10 to +10 percent (per axis)
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        # if mode is constant, use a cval between 0 and 255
        cval=255,
    )
], random_order=True)

x=X_train
y=y_train
aug=20

for i in range(aug):
	x_aug=seq.augment_images(x)
	X_train=np.append(X_train, x_aug, axis=0)
	y_train=np.append(y_train, y, axis=0)

In [ ]:
from numpy import asarray
print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)

print("X_val Shape: ", X_val.shape)
print("y_val Shape: ", y_val.shape)

print("y_test Shape: ", X_test.shape)

# print("X_train_aug Shape: ", X_train_aug.shape)
# print("y_train_aug Shape: ", y_train_aug.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers import LeakyReLU

def get_model():
    m1=Sequential()
    m1.add(BatchNormalization(input_shape = (160,160,3)))
    m1.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (160, 160, 3))) 
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Convolution2D(filters=6,kernel_size=4,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Convolution2D(filters=160,kernel_size=3,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Convolution2D(filters=160,kernel_size=2,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Flatten()) 
    m1.add(Dense(units=160,activation = 'relu'))
    m1.add(Dense(units = 64, activation = 'relu'))
    m1.add(Dense(units = 32, activation = 'relu'))
    m1.add(Dense(units = 350, activation = 'softmax'))
    m1.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
    return m1

In [ ]:
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=5, random_state = 2021, shuffle=True)
acc_list = [] 
# for train_index, val_index in kfold.split(X_train):
#     x_train_fold, x_val_fold = X_train[train_index], X_train[val_index]
#     y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
#     model = get_model()
#     model.fit(x_train_fold, y_train_fold, epochs = 30, validation_data = (x_val_fold, y_val_fold))

#     _, acc = model.evaluate(X_val,y_val)
#     acc_list.append(round(acc,4))

model.fit(X_train, y_train, epochs=30, validation_data=(X_val,y_val))

In [ ]:
# print(acc_list)

In [ ]:
predicted = model.predict(X_test, batch_size=350)
predicted = np.argmax(np.round(predicted),axis=1)
print("Prediction :",predicted)
print(y_test)
predicted=predicted.tolist()

In [ ]:
for i in range(700):
    predicted[i]+=1

y_test = map(int, y_test)
result = { name:value for name, value in zip(y_test, predicted) }
result = sorted(result.items())

print(result)

In [ ]:
import csv
with open('187180_이혜승_얼굴_2차_답안.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(result)